## Validate protein mutations with endstate corrections!!!

this should tell us if we are at all capable of closing cycles...

In [2]:
#!/usr/bin/env python
# coding: utf-8

# # Here, I document an attempt to validate a small set of protein mutations in vacuum and solvent with the following checks...

# 1. generate alanine dipeptide --> valine dipeptide in vac/solvent and conduct a forward _and_ reverse parallel tempering FEP calculation; the check passes if the forward free energy is equal to the reverse free energy within an error tolerance
# 2. generate alanine dipeptide --> valine dipeptide --> isoleucine dipeptide --> glycine dipeptide and attempt to close the thermodynamic cycle within an error tolerance

# In[ ]:


from __future__ import absolute_import

import networkx as nx
from perses.dispersed import feptasks
from perses.utils.openeye import *
from perses.utils.data import load_smi
from perses.annihilation.relative import HybridTopologyFactory
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from perses.rjmc.topology_proposal import TopologyProposal, TwoMoleculeSetProposalEngine, SystemGenerator,SmallMoleculeSetProposalEngine, PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
import simtk.openmm.app as app
import sys

from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState

import pymbar
import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import copy
import pickle
import mdtraj as md
from io import StringIO
from openmmtools.constants import kB
import logging
import os
import dask.distributed as distributed
import parmed as pm
from collections import namedtuple
from typing import List, Tuple, Union, NamedTuple
from collections import namedtuple
import random
#beta = 1.0/(kB*temperature)
import itertools
import os
from openeye import oechem
from perses.utils.smallmolecules import render_atom_mapping
from perses.tests.utils import validate_endstate_energies

ENERGY_THRESHOLD = 1e-6
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
from perses.tests.utils import validate_endstate_energies


# In[ ]:


from perses.samplers.multistate import HybridSAMSSampler, HybridRepexSampler
from openmmtools.multistate import MultiStateReporter, MultiStateSamplerAnalyzer
from openmmtools import mcmc, utils
from perses.annihilation.lambda_protocol import LambdaProtocol


# In[ ]:


def generate_atp(phase = 'vacuum'):
    """
    modify the AlanineDipeptideVacuum test system to be parametrized with amber14ffsb in vac or solvent (tip3p)
    """
    import openmmtools.testsystems as ts
    atp = ts.AlanineDipeptideVacuum(constraints = app.HBonds, hydrogenMass = 4 * unit.amus)

    forcefield_files = ['gaff.xml', 'amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
    
    if phase == 'vacuum':
        barostat = None
        system_generator = SystemGenerator(forcefield_files,
                                       barostat = barostat,
                                       forcefield_kwargs = {'removeCMMotion': False, 
                                                            'ewaldErrorTolerance': 1e-4, 
                                                            'nonbondedMethod': app.NoCutoff,
                                                            'constraints' : app.HBonds, 
                                                            'hydrogenMass' : 4 * unit.amus})
        atp.system = system_generator.build_system(atp.topology) #update the parametrization scheme to amberff14sb
        
    elif phase == 'solvent':
        barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
        system_generator = SystemGenerator(forcefield_files,
                                   barostat = barostat,
                                   forcefield_kwargs = {'removeCMMotion': False, 
                                                        'ewaldErrorTolerance': 1e-4, 
                                                        'nonbondedMethod': app.PME,
                                                        'constraints' : app.HBonds, 
                                                        'hydrogenMass' : 4 * unit.amus})
    
    if phase == 'solvent':
        modeller = app.Modeller(atp.topology, atp.positions)
        modeller.addSolvent(system_generator._forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
        solvated_topology = modeller.getTopology()
        solvated_positions = modeller.getPositions()

        # canonicalize the solvated positions: turn tuples into np.array
        atp.positions = unit.quantity.Quantity(value = np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit = unit.nanometers)
        atp.topology = solvated_topology

        atp.system = system_generator.build_system(atp.topology)
    
    
    return atp, system_generator


# In[ ]:


def generate_top_pos_sys(topology, new_res, system, positions, system_generator):
    """generate point mutation engine, geometry_engine, and conduct topology proposal, geometry propsal, and hybrid factory generation"""
    #create the point mutation engine
    print(f"generating point mutation engine")
    point_mutation_engine = PointMutationEngine(wildtype_topology = topology,
                                                system_generator = system_generator,
                                                chain_id = '1', #denote the chain id allowed to mutate (it's always a string variable)
                                                max_point_mutants = 1,
                                                residues_allowed_to_mutate = ['2'], #the residue ids allowed to mutate
                                                allowed_mutations = [('2', new_res)], #the residue ids allowed to mutate with the three-letter code allowed to change
                                                aggregate = True) #always allow aggregation

    #create a geometry engine
    print(f"generating geometry engine")
    geometry_engine = FFAllAngleGeometryEngine(metadata=None, 
                                           use_sterics=False, 
                                           n_bond_divisions=100, 
                                           n_angle_divisions=180, 
                                           n_torsion_divisions=360, 
                                           verbose=True, 
                                           storage=None, 
                                           bond_softening_constant=1.0, 
                                           angle_softening_constant=1.0, 
                                           neglect_angles = False, 
                                           use_14_nonbondeds = True)

    #create a top proposal
    print(f"making topology proposal")
    topology_proposal, local_map_stereo_sidechain, new_oemol_sidechain, old_oemol_sidechain = point_mutation_engine.propose(current_system = system,
                                  current_topology = topology)

    #make a geometry proposal forward
    print(f"making geometry proposal from {list(topology.residues())[1].name} to {new_res}")
    forward_new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta)
    logp_reverse = geometry_engine.logp_reverse(topology_proposal, forward_new_positions, positions, beta)


    #create a hybrid topology factory
    f"making forward hybridtopologyfactory"
    forward_htf = HybridTopologyFactory(topology_proposal = topology_proposal,
                 current_positions =  positions,
                 new_positions = forward_new_positions,
                 use_dispersion_correction = False,
                 functions=None,
                 softcore_alpha = None,
                 bond_softening_constant=1.0,
                 angle_softening_constant=1.0,
                 soften_only_new = False,
                 neglected_new_angle_terms = [],
                 neglected_old_angle_terms = [],
                 softcore_LJ_v2 = True,
                 softcore_electrostatics = True,
                 softcore_LJ_v2_alpha = 0.85,
                 softcore_electrostatics_alpha = 0.3,
                 softcore_sigma_Q = 1.0,
                 interpolate_old_and_new_14s = False,
                 omitted_terms = None)
    
    if not topology_proposal.unique_new_atoms:
        assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
        assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
        vacuum_added_valence_energy = 0.0
    else:
        added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

    if not topology_proposal.unique_old_atoms:
        assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
        assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
        subtracted_valence_energy = 0.0
    else:
        subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential

#     self._vacuum_forward_neglected_angles = self._geometry_engine.forward_neglected_angle_terms
#     self._vacuum_reverse_neglected_angles = self._geometry_engine.reverse_neglected_angle_terms
#     self._vacuum_geometry_engine = copy.deepcopy(self._geometry_engine)
    
    zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)
    print(f"zero state error : {zero_state_error}")
    print(f"one state error : {one_state_error}")
    
    return topology_proposal, forward_new_positions, forward_htf, local_map_stereo_sidechain, old_oemol_sidechain, new_oemol_sidechain, added_valence_energy, subtracted_valence_energy

    


def generate_fully_connected_perturbation_graph(dipeptides = ['ALA', 'SER', 'CYS', 'THR'], phase = 'vacuum'):
    # generate a fully connected solvation energy graph for the dipeptides specified...
    graph = nx.DiGraph()
    for dipeptide in dipeptides:
        graph.add_node(dipeptide)
    
    #now for edges...
    for i in graph.nodes():
        for j in graph.nodes():
            if i != j:
                graph.add_edge(i, j)
    
    
    #start with ala
    atp, system_generator = generate_atp(phase = phase)
    
    #graph.nodes['ALA']['vac_sys_pos_top'] = (vac_atp.system, vac_atp.positions, vac_atp.topology)
    graph.nodes['ALA']['sys_pos_top'] = (atp.system, atp.positions, atp.topology)
    
    #turn ala into all of the other dipeptides
    for dipeptide in [pep for pep in dipeptides if pep != 'ALA']:
        for _phase, testcase, sys_gen in zip([phase], [atp], [system_generator]):
            top_prop, new_positions, htf, local_map_stereo_sidechain, old_oemol, new_oemol, added_e, subtracted_e =  generate_top_pos_sys(testcase.topology, dipeptide, testcase.system, testcase.positions, sys_gen)
            new_sys, new_pos, new_top = htf._new_system, htf._new_positions, top_prop._new_topology
            graph.nodes[dipeptide][f"{_phase}_sys_pos_top"] = (new_sys, new_pos, new_top)
            graph.edges[('ALA', dipeptide)][f'{_phase}_htf'] = htf
            graph.edges[('ALA', dipeptide)][f"map_oldmol_newmol"] = (local_map_stereo_sidechain, old_oemol, new_oemol)
            graph.edges[('ALA', dipeptide)][f"added_subtracted"] = (added_e, subtracted_e)

        
        
    #now we can turn all of the other states in to each other!!!
    for edge_start, edge_end in list(graph.edges()):
        if edge_start == 'ALA': #we already did ALA
            continue
        
        for _phase, sys_gen in zip([phase], [system_generator]):
            sys, pos, top = graph.nodes[edge_start][f"{_phase}_sys_pos_top"]
            top_prop, new_positions, htf, local_map_stereo_sidechain, old_oemol, new_oemol, added_e, subtracted_e = generate_top_pos_sys(top, edge_end, sys, pos, sys_gen)
            new_sys, new_pos, new_top = htf._new_system, htf._new_positions, top_prop._new_topology
            graph.nodes[edge_end][f"{_phase}_sys_pos_top"] = (new_sys, new_pos, new_top)
            graph.edges[(edge_start, edge_end)][f"{_phase}_htf"] = htf
            graph.edges[(edge_start, edge_end)][f"map_oldmol_newmol"] = (local_map_stereo_sidechain, old_oemol, new_oemol)
            graph.edges[(edge_start, edge_end)][f"added_subtracted"] = (added_e, subtracted_e)
            
    print(f"graph_edges: {graph.edges()}")
    
    return graph
        


# In[ ]:


#os.system(f"rm *.nc")

            
        
    

    


In [3]:
mapping_strength = 'default'
import pickle
phases = ['solvent']
from perses.utils.smallmolecules import render_atom_mapping
graph = generate_fully_connected_perturbation_graph(phase = 'solvent')
print(f"graph edges: {graph.edges()}")
for pair in graph.edges():
    for _phase in phases:
        print("Seralizing the system to ", f"{pair}_{_phase}" + ".xml")
        with open(f"{pair[0]}_{pair[1]}.{_phase}.{mapping_strength}_map.xml", 'w') as f:
            hybrid_system = graph.edges[pair][f"{_phase}_htf"]._hybrid_system
            f.write(openmm.openmm.XmlSerializer.serialize(hybrid_system))
        
        htf = graph.edges[pair][f"{_phase}_htf"]
        htf._topology_proposal._old_networkx_residue.remove_oemols_from_graph()
        htf._topology_proposal._new_networkx_residue.remove_oemols_from_graph()
        _map, oldmol, newmol = graph.edges[pair][f"map_oldmol_newmol"]
        render_atom_mapping(f"{pair[0]}_{pair[1]}.{mapping_strength}_map.png", oldmol, newmol, _map)
        with open(f"{pair[0]}_{pair[1]}.{_phase}.{mapping_strength}_map.pkl", 'wb') as f:
            pickle.dump(htf, f)

generating point mutation engine
generating geometry engine
making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 5
INFO:geometry:Atom index proposal order is [10, 15, 12, 11, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 10 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 37 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 54 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switc

making geometry proposal from ALA to SER


INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {}
INFO:geometry:neglected angle terms include []
INFO:geometry:log probability choice of torsions and atom order: -6.4457198193855785
INFO:geometry:creating platform, integrators, and contexts; setting growth parameter
INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 5 new atoms
INFO:geometry:	reduced angle potential = 0.40580354257806844.
INFO:geometry:	reduced angle potential = 1.6822902051033777.
INFO:geometry:	reduced angle potential = 1.7722974543105718.
INFO:geometry:	reduced angle potential = 0.29251912941862934.
INFO:geometry:	reduced angle potential = 0.4521847067084046.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce

added energy components: [('CustomBondForce', 1.1328021102335801), ('CustomAngleForce', 6.884965058105415), ('CustomTorsionForce', 10.607699573127729), ('CustomBondForce', -34.369485007840964)]


...

			HarmonicBondForce: 1.8041543659710229
			HarmonicAngleForce: 3.6843818662540575
			PeriodicTorsionForce: 22.16266465301152
			NonbondedForce: -870.8733485582923
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:-843.2221476730557
rp: -842.8094246382728
zero state error : 2.9771740628348198e-12
one state error : 9.485745522397337e-12
generating point mutation engine
generating geometry engine
making topology proposal
making geometry proposal from THR to CYS


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 10 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 37 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 54 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1636 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {}
INFO:geometry:neglected angle terms inclu

added energy components: [('CustomBondForce', 0.312289594363839), ('CustomAngleForce', 4.017513147025921), ('CustomTorsionForce', 10.20491540684886), ('CustomBondForce', 2.7303377070702934)]


INFO:geometry:	reduced angle potential = 0.16085073244885684.
INFO:geometry:	reduced angle potential = 0.04539628107391919.
INFO:geometry:	reduced angle potential = 0.06336434983064564.
INFO:geometry:	reduced angle potential = 1.4536232120192898.
INFO:geometry:	reduced angle potential = 0.07825648903262361.
INFO:geometry:	reduced angle potential = 0.8858976069823169.
INFO:geometry:	reduced angle potential = 0.04129387242817869.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in t

added energy components: [('CustomBondForce', 2.5760091526492084), ('CustomAngleForce', 7.017708563089978), ('CustomTorsionForce', 20.72762225229503), ('CustomBondForce', -94.08750493071497)]
			HarmonicBondForce: 2.6100405438623495
			HarmonicAngleForce: 7.173790940264358
			PeriodicTorsionForce: 29.653978839126154
			NonbondedForce: -900.8796909349691
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:-861.4418806117162
rp: -861.0291575769336
			CustomBondForce: 1.8647390448810859e-10
			HarmonicBondForce: 2.922330138039715
			CustomAngleForce: 0.05177605532145648
			HarmonicAngleForce: 11.139528031968824
			CustomTorsionForce: 4.921232087027492
			PeriodicTorsionForce: 34.93766215894754
			NonbondedForce: -953.0334366949173
			CustomNonbondedForce: 54.88408346702133
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:-844.1768247564045
rp: -843.7641017216218
			CustomBondForce: 1.8647390448810859e-10
			HarmonicBondForce: 2.9223

In [ ]:
_dict = graph.edges['ALA', 'PHE']

In [ ]:
_dict

In [ ]:
htf = _dict['vacuum_htf']

In [ ]:
#zero_state_error, one_state_error = validate_endstate_energies(_dict[f'{phase}_htf']._topology_proposal, _dict[f'{phase}_htf'], _dict['added_subtracted'][0], _dict['added_subtracted'][1], beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)

# Hooray!!!!!!!


In [ ]:
def create_new_pdb(topology, positions, output_pdb = 'test_new.pdb'):
    """
    create a pdb of the geometry proposal (only new system)
    """
    import mdtraj as md
    _positions =  np.array(positions.value_in_unit(unit.nanometer))
    print(_positions)
    traj = md.Trajectory(_positions, md.Topology.from_openmm(topology))
    traj.save(output_pdb)

In [ ]:
create_new_pdb(topology = htf._topology_proposal._old_topology, positions = htf._old_positions, output_pdb = 'ALA_old.pdb')
create_new_pdb(topology = htf._topology_proposal._new_topology, positions = htf._new_positions, output_pdb = 'PHE_good.pdb')

In [ ]:
import mdtraj as md

In [ ]:
top = md.Topology().from_openmm(htf._topology_proposal._new_topology)

In [ ]:
table, bonds = top.to_dataframe()

In [ ]:
table

In [ ]:
bonds